In [17]:
import numpy as np
import cv2
import pathlib
import os
from scipy.linalg import null_space
import sympy
import time

class Camera:
    
    def __init__(self, val):
        self.img_num = val # カメラ番号（int;コンストラクタ）
        
        f = 8000/3
        cx = 1920/2
        cy = 1080/2
        A = np.zeros((3,3))
        A[0,0] = f
        A[0,2] = cx
        A[1,1] = f
        A[1,2] = cy
        A[2,2] = 1
        
        self.A = A # 内部パラメータ(ndarray)後から更新

    def img_load(self):
        folder_path = "image"
        file_path = os.path.join(folder_path, str(self.img_num) + ".png")
        img = cv2.imread(file_path, 1)# BGRで読み込み
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.flip(img, 1)
        self.img = img # 画像(ndarray)
        
    def contour_extraction(self):
        
        color_arr = np.array([[255,0,0],[0,255,0],[0,0,255],
                             [255,255,0],[255,0,255],[0,255,255],
                             [127,127,127],[127,0,127],[0,127,127]],dtype = np.int16)
        masks = np.ones((self.img.shape[0], self.img.shape[1], 9), dtype=np.uint8)
        
        for i, color in enumerate(color_arr):
            lower = np.clip(color, 0, 255)
            upper = np.clip(color, 0, 255)
            img_mask = cv2.inRange(self.img, lower, upper)
            masks[:,:,i] = img_mask
        
        #self.masks = masks # 色ごとのマスク(nd.array)
        
        contour_list = []

        # 色ごとに輪郭（閉曲線）を抽出
        for i in range(masks.shape[2]):
            contours, hierarchy = cv2.findContours(masks[:,:,i],cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
            contour_list.append(contours)
        self.contour_list = contour_list # 輪郭のリスト(list,ndarray)

        #self.frag_list = contours2fragments(self.contour_list) # フラグメントのリスト(list,ndarray)

    def para_load(self):

        folder_path = pathlib.Path("view_mat")
        file_path = os.path.join(folder_path, str(self.img_num)+".csv")
        self.Rt = np.loadtxt(file_path, delimiter="\t")
        self.P = np.dot(self.A, self.Rt[0:3,0:4])
        
        folder_path = pathlib.Path("cam_pos_rot")
        file_path = os.path.join(folder_path, str(self.img_num)+".csv")
        self.cam_world_cood = np.loadtxt(file_path, delimiter=',')[0]

In [18]:
# 曲線分割
def split_list(contour_length, max_frag_len=100, min_frag_len=30, min_overrap=10):
    
    # 輪郭のフラグメントの位置を指定(最小40 pixl)
    if contour_length > max_frag_len:
        pass
    
    elif contour_length < min_frag_len:
        return None
    
    elif contour_length == min_frag_len:
        return [[0,min_frag_len-1]]
    
    else:
        max_frag_len = contour_length
    
    step0 = np.random.randint(min_frag_len, max_frag_len) # 一つ目のフラグメントの長さ（40から100）
    sep_list = [[0,step0]]
    back = np.random.randint(min_overrap, step0-1) # フラグメントを重ねるために戻す分を決める（最小10 pixl）
    next_start = step0 - back
    
    while True:
        
        # 戻った分(back)より進む
        if back+1 > min_frag_len:
            step = np.random.randint(back+1, max_frag_len)
        else:
            step = np.random.randint(min_frag_len, max_frag_len)

        full_length = next_start + step
        sept = [next_start, full_length]
        sep_list.append(sept)
        back = np.random.randint(10, step-1)
        next_start = full_length - back

        # 終了判定
        if full_length > contour_length:
            break
    
    # 超過した分戻す（長さはそのまま）
    difference = sep_list[-1][1] - (contour_length-1)
    sep_list[-1][0] -= difference
    sep_list[-1][1] -= difference
    
    return sep_list


def contours_split(contour):
    
    #contour.shape == (N, 2)
    contour_length = contour.shape[0]
    sp_list = split_list(contour_length)
    
    if sp_list == None:
        return None
    
    frag_list = []
    # 位置のリスト通りにスライス
    for sp in sp_list:
        #print(sp)
        frag_list.append(contour[sp[0]:sp[1],:])

    return frag_list


def all_fraged(contours_list):
    
    # 輪郭のリストからフラグメントのリストを得る
    frags_list = []

    #for i in contours_list:
    #temp_list = []
    frags = []
    for j in contours_list:
        temp_frags = contours_split(j.squeeze())
            
        if temp_frags != None:
            frags += temp_frags

    #if frags != []:
    #    frags_list.append(frags)
    
    return frags

def frag_list_fraged(frags_list):# frags_list[色][輪郭][sep][座標]
    img_frag_list = []
    for frag in frags_list:
        color_frag = all_fraged(frag)
        img_frag_list.append(color_frag)
    return img_frag_list

In [19]:
# camera対応
def dim3_distance(vec1, vec2):
    return sum((vec1-vec2)**2)

def camera_correspondence(cam_list):
    vec_list = []
    for i,cam in enumerate(cam_list):
        cam_list[i].para_load()
        vec_list.append(cam_list[i].cam_world_cood)

    pair_list = []
    for i, vec1 in enumerate(vec_list):
        for j, vec2 in enumerate(vec_list):
            if i == j or i > j :
                continue
            elif dim3_distance(vec1,vec2) < 2:
                pair_list.append((i,j))
    
    return pair_list

In [20]:
# epipole取得

def SS_mat(vec3):
    vec3 = np.squeeze(vec3)
    SS_mat = np.zeros((3, 3))
    SS_mat[0,1] = -vec3[2]
    SS_mat[0,2] = vec3[1]
    SS_mat[1,0] = vec3[2]
    SS_mat[1,2] = -vec3[0]
    SS_mat[2,0] = -vec3[1]
    SS_mat[2,1] = vec3[0]
    return SS_mat

def FF_mat(A1, A2, Rt1, Rt2):
    P1 = np.dot(A1, Rt1[0:3,0:4])
    P2 = np.dot(A2, Rt2[0:3,0:4])
    cam_pos1 = -np.dot(Rt1[0:3,0:3].T, Rt1[0:3,3])
    cam_pos1 = np.array([cam_pos1[0], cam_pos1[1], cam_pos1[2], 1])
    epipole2 = np.dot(P2, cam_pos1)
    cam_pos2 = -np.dot(Rt2[0:3,0:3].T, Rt2[0:3,3])
    cam_pos2 = np.array([cam_pos2[0], cam_pos2[1], cam_pos2[2], 1])
    epipole1 = np.dot(P1, cam_pos2)
    return epipole1, epipole2, np.dot(SS_mat(epipole2), np.dot(P2, np.linalg.pinv(P1)))

def contour_disassembly(contour_list):
    con_dis = []
    for i in range(len(contour_list)):
        if contour_list[i] == []:
            continue
        which_dis = np.concatenate(contour_list[i])
        con_dis.append(which_dis)
        
    return np.concatenate(con_dis)

In [21]:
# エピポーラ関係
def epilines_para(frags, F):
    
    lines_list = []
    for color in frags:
        temp_color_list = []
        for frag in color:
            frag_lines = cv2.computeCorrespondEpilines(frag.reshape(-1,1,2), 1,F) # ndarray(フラグメントの座標数,1,3)
            temp_color_list.append(frag_lines)

        lines_list.append(temp_color_list)
    
    return lines_list


def para2line(parameter):
    
    # 一つのパラメータが渡された時を想定
    line_coode = np.zeros((1920,2), dtype=np.int64)
    para = np.squeeze(parameter)# 3次ベクトル
    for x in range(1920):
        y = int((-para[0]*x - para[2])/para[1])
        line_coode[x,0] = x
        line_coode[x,1] = y
    
    return line_coode


def epiline_cal(frag_paras):
    # 全ての色に対するエピポーラ線の帯の計算
    # lines[色][フラグメント][線][座標]
    lines = []
    for color in frag_paras:
        temp_color = []
        
        for frag in color:
            temp_line = []

            for point in frag:
                line = para2line(point)
                temp_line.append(line)

            temp_color.append(temp_line)
        lines.append(temp_color)
        
    return lines


def frags_vs_line(img2_frags, frag_epiline):
    
    # frag_epiline shape(1920, 2)
    surport = np.zeros(len(img2_frags))
    for i in frag_epiline:
        for j, frag in enumerate(img2_frags):
            if i in frag:
                surport[j] += 1
    
    return surport


def pair_frag_idx(img2_frags, frag_epilines):
    surport = np.zeros(len(img2_frags))
    for epi in frag_epilines:
        surport += frags_vs_line(img2_frags, epi)
        
    return surport, np.argmax(surport)


In [22]:
def normalization(vec3):
    return vec3[0]/vec3[2], vec3[1]/vec3[2]

In [23]:
# 再構築
def nom_F(F):
    return (1/sum(sum(F**2))**(1/2))*F

def cover_mat(x1, y1, x2, y2):
    return np.array([[x1**2+x2**2, x2*y2, x2, x1*y1, 0, 0, x1, 0, 0],
                    [x2*y2, x1**2+y2**2, y2, 0, x1*y1, 0, 0, x1, 0],
                    [x2, y2, 1, 0, 0, 0, 0, 0, 0],
                    [x1*y1, 0, 0, y1**2+x2**2, x2*y2, x2, y1, 0, 0],
                    [0, x1*y1, 0, x2*y2, y1**2+y2**2, y2, 0, y1, 0],
                    [0, 0, 0, x2, y2, 1, 0, 0, 0],
                    [x1, 0, 0, y1, 0, 0, 1, 0, 0],
                    [0, x1, 0, 0, y1, 0, 0, 1, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0]])

def min_dist(F, pt1, pt2):# pt1が画像2上の点，pt2が画像1上の点
    S0 = 10**10
    x1_ori = pt1[0]
    y1_ori = pt1[1]
    x2_ori = pt2[0]
    y2_ori = pt2[1]
    
    x1 = pt1[0]
    y1 = pt1[1]
    x2 = pt2[0]
    y2 = pt2[1]
    
    x1_tilda = 0
    y1_tilda = 0
    x2_tilda = 0
    y2_tilda = 0
    thita = nom_F(F).flatten()
    it = 0
    while True:
        V_eps = cover_mat(x1, y1, x2, y2)
        eps_ast = np.array([x1*x2 + x2*x1_tilda + x2*x2_tilda,
                           x1*y2 + y2*x1_tilda + x2*y2_tilda,
                           x1 + x1_tilda,
                           y1*x2 + x2*y1_tilda + y1 * x2_tilda,
                           y1* y2 + y2*y1_tilda + y1*y2_tilda,
                           y1 + y1_tilda,
                           x2 + x2_tilda,
                           y2 + y2_tilda,
                           1])
        
        x1_y1_tilda = np.dot(eps_ast, thita) * np.dot(np.array([[thita[0], thita[1], thita[2]], [thita[3], thita[4], thita[5]]]), np.array([x2, y2, 1])) / np.dot(thita, np.dot(V_eps, thita))
        x2_y2_tilda = np.dot(eps_ast, thita) * np.dot(np.array([[thita[0], thita[3], thita[6]], [thita[1], thita[4], thita[7]]]), np.array([x1, y1, 1])) / np.dot(thita, np.dot(V_eps, thita))
        
        x1_tilda = x1_y1_tilda[0]
        y1_tilda = x1_y1_tilda[1]
        x2_tilda = x2_y2_tilda[0]
        y2_tilda = x2_y2_tilda[1]
        
        x1 = x1_ori - x1_tilda
        y1 = y1_ori - y1_tilda
        x2 = x2_ori - x2_tilda
        y2 = y2_ori - y2_tilda
        
        S = x1_tilda**2 + y1_tilda**2 + x2_tilda**2 + y2_tilda**2
        
        if abs(S0 - S) < 0.00001:
            break

        elif it == 20:
            break

        else:
            S0 = S
            it += 1

    return np.array((x1, y1)), np.array((x2, y2))

def Ps(P, pt):
    a = P[0,0] - pt[0]*P[2,0]
    b = P[0,1] - pt[0]*P[2,1]
    c = P[0,2] - pt[0]*P[2,2]
    d = P[0,3]
    e = pt[0]*P[2,3]
    f = P[1,0] - pt[1]*P[2,0]
    g = P[1,1] - pt[1]*P[2,1]
    h = P[1,2] - pt[1]*P[2,2]
    i = P[1,3]
    j = pt[1]*P[2,3]
    return a, b, c, d, e, f, g, h, i, j

def tri(P1, P2, pt1, pt2):
    #x = sympy.Symbol('x')
    #y = sympy.Symbol('y')
    #z = sympy.Symbol('z')
    a1, b1, c1, d1, e1, f1, g1, h1, i1, j1 = Ps(P1, pt1)
    a2, b2, c2, d2, e2, f2, g2, h2, i2, j2 = Ps(P2, pt2)
    T = np.array([[a1, b1, c1],
                 [f1, g1, h1],
                 [a2, b2, c2],
                 [f2, g2, h2]])
    p = np.array([[d1-e1],
                 [i1-j1],
                 [d2-e2],
                 [i2-j2]])
    T_inv = np.linalg.pinv(T)
    result_pt = np.dot(T_inv, -p)
    return result_pt

    #expr1 = a1*x + b1*y + c1*z + d1 - e1
    #expr2 = f1*x + g1*y + h1*z + i1 - j1
    #expr3 = a2*x + b2*y + c2*z + d2 - e2
    #expr4 = f2*x + g2*y + h2*z + i2 - j2
    #print(expr1.subs(sympy.solve([expr2, expr3, expr4])))
    #result = sympy.solve([expr2, expr3, expr4])
    
    #return np.array([result[x], result[y], result[z]])

pt1 = [1920-741,758] # 0
pt2 = [1920-858,834] # 2
F = cam_pairs_F[(0,2)] #0to2
pt2, pt1 = min_dist(F, pt2, pt1)
print(pt1,pt2)

pt1 = np.array([pt1[0],pt1[1],1]) # 0
pt2 = np.array([pt2[0],pt2[1],1]) # 2

np.dot(np.dot(pt2, F),pt1)

tri(cam_list[0].P, cam_list[2].P, pt1, pt2)

In [24]:
import matplotlib.pyplot as plt

In [25]:
cam_list = [Camera(i) for i in range(24)]
for i in range(len(cam_list)):
    cam_list[i].img_load()
    cam_list[i].contour_extraction()
    cam_list[i].para_load()
#cam_pairs = camera_correspondence(cam_list)

In [26]:
# エピポール取得
cam_pairs = camera_correspondence(cam_list)

epipole_dict = {i:[] for i in range(len(cam_list))}
cam_pairs_F = {}
for i in cam_pairs:
    epipole1, epipole2, F = FF_mat(cam_list[i[0]].A, cam_list[i[1]].A, cam_list[i[0]].Rt, cam_list[i[1]].Rt)
    epipole_dict[i[0]].append(normalization(epipole1))
    epipole_dict[i[1]].append(normalization(epipole2))
    cam_pairs_F[i] = F

In [27]:
#エピポールと輪郭上の点を結んだ直線と，x軸のなす角
def gene(angles):
    # 正規化
    B = list(map(lambda y:y-min(angles),angles))
    return list(map(lambda y: (y-min(B))/(max(B)-min(B)), B))

def epipole_angle(img_num, epipole_dict):
    cam = cam_list[img_num]
    cam.img_load()
    cam.contour_extraction()
    angle_list = []
    
    for epi in epipole_dict[img_num]:
        epi_angle_list = []
        epi_x = epi[0]
        epi_y = epi[1]
        for color in cam.contour_list:
            color_angle_list = []
            for contour in color:
                contour_angle_list= []
                for cood in contour:
                    x = cood[0][0]
                    y = cood[0][1]
                    tilt = (y-epi_y)/(x-epi_x)
                    angle = np.arctan(tilt)
                    contour_angle_list.append(angle)
                contour_angle_list = gene(contour_angle_list)
                color_angle_list.append(contour_angle_list)
            epi_angle_list.append(color_angle_list)
        angle_list.append(epi_angle_list)
    return angle_list
    
def expand(idx_l, list_length):
    del_list = []
    for i in idx_l:
        if np.isnan(i):
            continue
        if i-2 < 0:
            del_list.append(list_length + i-2)
        else:
            del_list.append(i-2)
        
        if i-1 < 0:
            del_list.append(list_length + i-1)
        else:
            del_list.append(i-1)
        
        del_list.append(i)
        
        if i+1 > list_length-1:
            del_list.append(i+1-list_length)
        else:
            del_list.append(i+1)
        if i+2 > list_length-1:
            del_list.append(i+2-list_length)
        else:
            del_list.append(i+2)
    return sorted(list(set(del_list)))

def differential(angles):
    # エピポーラ線に平行な接線をもつ点(前後方微分の正負を比べたほうが良い)
    del_idx = []
    for i in range(len(angles)):
        if np.isnan(angles[i]):
            continue
        if i == len(angles)-1:
            if np.sign(angles[i]-angles[i-1]) != np.sign(angles[0]-angles[i]): #or abs(angles[0]-angles[i-1])/2 < 0.001:
                del_idx.append(i)
        else:
            if np.sign(angles[i]-angles[i-1]) != np.sign(angles[i+1]-angles[i]):# or abs(angles[i+1]-angles[i-1])/2 < 0.001:
                del_idx.append(i)
    #del_idx = expand(del_idx, len(angles))
    
    return del_idx

def marge_del(epi_del_list):
    im_del_list = []
    for i, col in enumerate(epi_del_list[0]):
        color_list = []
        for j, con in enumerate(col):
            color_list.append(list(set(epi_del_list[0][i][j] + epi_del_list[1][i][j])))
        im_del_list.append(color_list)
    return im_del_list

def all_D(angles_list):
    # 画像1枚に対して削除リストを作成
    all_del_list = []
    for epi in angles_list:
        epi_del_list = []
        for color in epi:
            color_del_list = []
            for contour in color:
                #if len(contour)<40:
                #    continue
                del_idx = differential(contour)
                color_del_list.append(del_idx)
            epi_del_list.append(color_del_list)
        all_del_list.append(epi_del_list)
    all_del_list = marge_del(all_del_list)
    return all_del_list

In [28]:
# エピポーラ線との接線で分割
def separate(contour, del_idx):
    # 一つの輪郭に対し削除リストから削除
    start = 0
    newArray = []
    for d in del_idx:
        if contour[start:d] != []:
            if contour[start:d].size != 0:
                newArray.append(contour[start:d])
        start = d+1

    if contour[start:].size != 0:
        newArray.append(contour[start:])
    return newArray

def all_sep(con_list, del_list):
    n_list = []
    for col, del_col in zip(con_list, del_list):
        n_col_list = []
        for con, del_con in zip(col, del_col) :
            n_con = separate(con, del_con)
            for frag in n_con:
                n_col_list.append(frag)
        n_list.append(n_col_list)
    return n_list

In [29]:
# フラグメントのリストを作る
for i in range(len(cam_list)):
    im_del_list = all_D(epipole_angle(i, epipole_dict))# im_del_list[color][contour][del_idx]
    newCon = all_sep(cam_list[i].contour_list, im_del_list)# newCon[color][fragment][coordination]
    cam_list[i].frag_list = frag_list_fraged(newCon)

C:\Users\murata\anaconda3\envs\ana37\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
C:\Users\murata\anaconda3\envs\ana37\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  import sys


In [33]:
# 衝突判定
def para2cood_S(para_list):
    return np.array([[0, -c/b] for a,b,c in para_list], dtype=np.int64)

def para2cood_F(para_list):
    return np.array([[1920, -(1920*a+c)/b] for a,b,c in para_list], dtype=np.int64)

def all_pa2co(para_list):
    epi_cood_S = []
    epi_cood_F = []
    for color in para_list:
        color_list_S = []
        color_list_F = []
        for frag in color:
            S_cood = para2cood_S(frag.squeeze())
            F_cood = para2cood_F(frag.squeeze())
            color_list_S.append(S_cood)
            color_list_F.append(F_cood)
        epi_cood_S.append(color_list_S)
        epi_cood_F.append(color_list_F)
    return epi_cood_S, epi_cood_F # epi_cood[color][frag]

def get_frag_cood(frag_list):
    cood_S = []
    cood_F = []
    for color in frag_list:
        col_S = np.array([frag[0] for frag in color])
        col_F = np.array([frag[-1] for frag in color])
        cood_S.append(col_S)
        cood_F.append(col_F)
    return cood_S, cood_F # cood_S[color][frag]

def coll_t1_t2(epi_cood_S, epi_cood_F, cood_S, cood_F):
    epi_cood_S_bro = np.array([np.broadcast_to(aa, (len(cood_S), 2)) for aa in epi_cood_S])
    epi_cood_F_bro = np.array([np.broadcast_to(aa, (len(cood_S), 2)) for aa in epi_cood_F])
    v = cood_S - epi_cood_S_bro
    v2 = cood_F - cood_S
    v1 = epi_cood_F_bro - epi_cood_S_bro
    t1 = np.cross(v, v2)/np.cross(v1, v2)
    t2 = np.cross(v, v1)/np.cross(v1, v2)
    return t1, t2

def coll_det(t1, t2):
    t1_t = np.array((t1 <= 1) & (t1 > 0),dtype=np.int16)
    t2_t = np.array((t2 <= 1) & (t2 > 0),dtype=np.int16)
    count_c = np.array(t1_t + t2_t == 2, dtype=np.int64)
    surport_idx = np.argmax(np.sum(count_c,axis=0))
    return surport_idx

def make_piar_list(epi_cood_S, epi_cood_F, cood_S, cood_F):
    img_list=[]
    for epi_S_col, epi_F_col, S_col, F_col in zip(epi_cood_S, epi_cood_F, cood_S, cood_F):
        color_list = []
        if len(epi_S_col) == 0 or len(S_col) == 0:
            img_list.append(color_list)
            continue
        for epi_S_frag, epi_F_frag in zip(epi_S_col, epi_F_col):
            t1, t2 = coll_t1_t2(epi_S_frag, epi_F_frag, S_col, F_col)
            surport_idx = coll_det(t1, t2)
            color_list.append(surport_idx)
        img_list.append(color_list)
    return img_list

In [34]:
pair_list = {}
for pair in cam_pairs_F:
    F = cam_pairs_F[pair]
    frags_para12 = epilines_para(cam_list[pair[0]].frag_list, F) # frags_para[色][frag]
    frags_para21 = epilines_para(cam_list[pair[1]].frag_list, F.T)
    
    cood_S, cood_F = get_frag_cood(cam_list[pair[1]].frag_list)
    epi_cood_S, epi_cood_F = all_pa2co(frags_para12)
    img_list1 = make_piar_list(epi_cood_S, epi_cood_F, cood_S, cood_F)
    
    cood_S, cood_F = get_frag_cood(cam_list[pair[0]].frag_list)
    epi_cood_S, epi_cood_F = all_pa2co(frags_para21)
    img_list2 = make_piar_list(epi_cood_S, epi_cood_F, cood_S, cood_F)
    
    pair_list[((pair[0],pair[1]), "F")] = img_list1
    pair_list[((pair[0],pair[1]), "R")] = img_list2

0.0019981861114501953
0.0
0.0020051002502441406
0.0009589195251464844
0.0030362606048583984
0.0
0.002000093460083008
0.0
0.0020003318786621094
0.0
0.002000570297241211
0.0
0.002003192901611328
0.0
0.0019979476928710938
0.0009615421295166016
0.0010399818420410156
0.0009582042694091797
0.0020384788513183594
0.0
0.002965211868286133
0.0
0.0020346641540527344
0.0
0.0009634494781494141
0.0009999275207519531
0.0010366439819335938
0.0009622573852539062
0.0020372867584228516
0.0
0.001965045928955078
0.0
0.002001047134399414
0.0
0.0029993057250976562
0.0
0.002000570297241211
0.0
0.001999378204345703
0.0009615421295166016
0.0020380020141601562
0.0009636878967285156
0.0020399093627929688
0.0
0.001997709274291992
0.0009610652923583984
0.0020418167114257812
0.0
0.001995563507080078
0.0009624958038330078
0.0020380020141601562
0.0
0.002001047134399414
0.0009622573852539062
0.001035928726196289
0.0
0.0010013580322265625
0.0
0.002042055130004883
0.0009555816650390625
0.002041339874267578
0.0
0.00199675

0.0009622573852539062
0.002042055130004883
0.0
0.0019974708557128906
0.0
0.0019996166229248047
0.0
0.001967906951904297
0.0
0.0019941329956054688
0.0010020732879638672
0.0019998550415039062
0.0010006427764892578
0.0020003318786621094
0.0
0.0009989738464355469
0.0010004043579101562
0.0020008087158203125
0.0
0.001999378204345703
0.0
0.0009961128234863281
0.0009999275207519531
0.0010395050048828125
0.0
0.002003192901611328
0.0009567737579345703
0.002037525177001953
0.0009648799896240234
0.003039836883544922
0.0
0.0019943714141845703
0.0009634494781494141
0.002001047134399414
0.0
0.0030395984649658203
0.0
0.001989603042602539
0.0
0.0010371208190917969
0.0
0.0019998550415039062
0.0009617805480957031
0.002052783966064453
0.0009467601776123047
0.0020744800567626953
0.0009276866912841797
0.0010592937469482422
0.0009391307830810547
0.0010690689086914062
0.0
0.001970052719116211
0.0009605884552001953
0.0020699501037597656
0.0
0.0019996166229248047
0.0010030269622802734
0.0019986629486083984
0.0


0.0009300708770751953
0.0010411739349365234
0.0009760856628417969
0.0010204315185546875
0.0
0.0019991397857666016
0.0
0.0019643306732177734
0.0010683536529541016
0.002001047134399414
0.0
0.002003908157348633
0.0009255409240722656
0.0010001659393310547
0.0009982585906982422
0.0020008087158203125
0.0
0.002070188522338867
0.0
0.0009300708770751953
0.0010030269622802734
0.000997304916381836
0.0
0.0020678043365478516
0.0
0.0009641647338867188
0.0009670257568359375
0.0010771751403808594
0.0009326934814453125
0.0020589828491210938
0.0
0.0009319782257080078
0.0010581016540527344
0.000942230224609375
0.0010688304901123047
0.0019986629486083984
0.0
0.0009307861328125
0.0010037422180175781
0.0010654926300048828
0.0
0.0020003318786621094
0.0009322166442871094
0.0010673999786376953
0.0
0.0010678768157958984
0.0009322166442871094
0.0010678768157958984
0.0
0.0009982585906982422
0.001018524169921875
0.0010495185852050781
0.0009307861328125
0.001001119613647461
0.0
0.0009999275207519531
0.0
0.000931739

0.001998424530029297
0.0
0.0020384788513183594
0.0
0.0009634494781494141
0.0009982585906982422
0.0010585784912109375
0.0
0.0019428730010986328
0.0
0.0020592212677001953
0.0
0.0009386539459228516
0.0010223388671875
0.0009791851043701172
0.0
0.0009999275207519531
0.0
0.0010006427764892578
0.0
0.0020589828491210938
0.0
0.0009500980377197266
0.0
0.0010290145874023438
0.0
0.0009717941284179688
0.0009996891021728516
0.0010447502136230469
0.0
0.0009539127349853516
0.0
0.0009677410125732422
0.0009996891021728516
0.0010280609130859375
0.0
0.0009708404541015625
0.00099945068359375
0.00102996826171875
0.0
0.001971721649169922
0.0
0.00099945068359375
0.0010004043579101562
0.0
0.0009999275207519531
0.0010290145874023438
0.0
0.0020325183868408203
0.0
0.001970052719116211
0.0
0.0009684562683105469
0.0010004043579101562
0.0010325908660888672
0.0
0.0009667873382568359
0.0
0.0020294189453125
0.0
0.0009708404541015625
0.0009984970092773438
0.0010273456573486328
0.0
0.0020020008087158203
0.0
0.00200986862

0.0
0.0009617805480957031
0.0
0.0010437965393066406
0.0
0.0010607242584228516
0.0
0.00093841552734375
0.0010142326354980469
0.0009875297546386719
0.0
0.002069711685180664
0.0009288787841796875
0.0010693073272705078
0.0009558200836181641
0.002045869827270508
0.0
0.001970529556274414
0.0
0.001961946487426758
0.0
0.0020742416381835938
0.0
0.0029249191284179688
0.0
0.00099945068359375
0.001031637191772461
0.0009756088256835938
0.0009915828704833984
0.0020623207092285156
0.0
0.001986265182495117
0.0009510517120361328
0.002007007598876953
0.0
0.0010027885437011719
0.0
0.0009987354278564453
0.0009996891021728516
0.0010008811950683594
0.0
0.0009989738464355469
0.0
0.001999378204345703
0.0
0.0009996891021728516
0.0
0.002000570297241211
0.0
0.0020003318786621094
0.0
0.002000570297241211
0.0
0.0
0.0
0.0009734630584716797
0.0009996891021728516
0.0010001659393310547
0.0
0.003000497817993164
0.0
0.002000570297241211
0.0009989738464355469
0.0020017623901367188
0.0
0.001998424530029297
0.0
0.000999212

...

0.0010001659393310547
0.0
0.0020003318786621094
0.0
0.0009999275207519531
0.0
0.0009999275207519531
0.0
0.0009996891021728516
0.0
0.0009996891021728516
0.0
0.0010004043579101562
0.0010008811950683594
0.0009996891021728516
0.0
0.0009989738464355469
0.0
0.0009999275207519531
0.0
0.0020008087158203125
0.0
0.0009999275207519531
0.0
0.0
0.0
0.0010001659393310547
0.0
0.0010001659393310547
0.0
0.0010001659393310547
0.0
0.0009989738464355469
0.0
0.0010008811950683594
0.0
0.0009999275207519531
0.0010001659393310547
0.00099945068359375
0.0
0.0010006427764892578
0.0
0.0009996891021728516
0.0
0.0009996891021728516
0.0
0.0010006427764892578
0.0
0.0009996891021728516
0.00099945068359375
0.0010006427764892578
0.0
0.00099945068359375
0.0
0.0010004043579101562
0.0009999275207519531
0.0009999275207519531
0.0
0.00099945068359375
0.0
0.0010004043579101562
0.0
0.00099945068359375
0.0
0.0010001659393310547
0.0
0.0010001659393310547
0.0
0.0009999275207519531
0.0
0.00099945068359375
0.0
0.0010004043579101562


0.0010004043579101562
0.0
0.0020008087158203125
0.0
0.0009996891021728516
0.0
0.0009989738464355469
0.0
0.0009987354278564453
0.0
0.0010006427764892578
0.0009999275207519531
0.0010018348693847656
0.0
0.001997709274291992
0.0
0.0
0.0009999275207519531
0.0010001659393310547
0.0
0.0010004043579101562
0.0
0.0010008811950683594
0.0010008811950683594
0.0
0.0
0.0010001659393310547
0.0
0.0009982585906982422
0.0010001659393310547
0.00099945068359375
0.0
0.0010001659393310547
0.0
0.0
0.0010006427764892578
0.0010006427764892578
0.0
0.0010006427764892578
0.0
0.0
0.0
0.0020012855529785156
0.0
0.0009996891021728516
0.0
0.0009996891021728516
0.0
0.0009989738464355469
0.0
0.0010075569152832031
0.0
0.0019927024841308594
0.0
0.0009996891021728516
0.0
0.0010004043579101562
0.0010004043579101562
0.0009992122650146484
0.0
0.0010013580322265625
0.0
0.000997781753540039
0.0
0.0010008811950683594
0.0
0.0009992122650146484
0.0010013580322265625
0.0009987354278564453
0.0
0.0009999275207519531
0.0
0.002000331878

0.0010001659393310547
0.0030002593994140625
0.0
0.002000093460083008
0.0009996891021728516
0.002000570297241211
0.0
0.002998828887939453
0.0
0.0020012855529785156
0.0
0.0020003318786621094
0.0
0.0019996166229248047
0.0
0.002000093460083008
0.0010001659393310547
0.0019998550415039062
0.0
0.0020020008087158203
0.0
0.002000093460083008
0.0009992122650146484
0.0009987354278564453
0.0
0.002000093460083008
0.0
0.0009996891021728516
0.0
0.002001047134399414
0.0010004043579101562
0.0019991397857666016
0.0
0.0020003318786621094
0.0009989738464355469
0.002000570297241211
0.0010004043579101562
0.001999378204345703
0.0
0.0019998550415039062
0.0
0.0019998550415039062
0.0
0.002000093460083008
0.0010008811950683594
0.0019996166229248047
0.0
0.0020003318786621094
0.0009989738464355469
0.0010008811950683594
0.0009999275207519531
0.0010001659393310547
0.0010037422180175781
0.0019958019256591797
0.0
0.0019998550415039062
0.0
0.0029997825622558594
0.0
0.0010006427764892578
0.0010001659393310547
0.00199890

0.001001596450805664
0.0009987354278564453
0.0
0.002000570297241211
0.0
0.002000570297241211
0.0
0.001998424530029297
0.0010008811950683594
0.0019996166229248047
0.0
0.002000570297241211
0.0010006427764892578
0.0019989013671875
0.0010006427764892578
0.002999544143676758
0.0
0.003000020980834961
0.0
0.003002166748046875
0.0
0.002001523971557617
0.0009975433349609375
0.001998424530029297
0.0
0.003001689910888672
0.0
0.001999378204345703
0.0
0.0029997825622558594
0.0
0.002000093460083008
0.0009999275207519531
0.0019991397857666016
0.0
0.002001523971557617
0.0009996891021728516
0.0009996891021728516
0.0010001659393310547
0.001999378204345703
0.0
0.0020003318786621094
0.0
0.00099945068359375
0.0010004043579101562
0.0019998550415039062
0.0
0.0009992122650146484
0.0010004043579101562
0.002000093460083008
0.0
0.002000093460083008
0.0
0.0020003318786621094
0.0009999275207519531
0.0009996891021728516
0.0
0.0030002593994140625
0.0
0.002000093460083008
0.0
0.0010001659393310547
0.00099897384643554

KeyboardInterrupt: 

In [32]:
# 線と点の衝突判定
def PL_coll(pair, pair_list, cam_list):
    F = cam_pairs_F[pair[0]]
    if pair[1] == "F":
        frags_para = epilines_para(cam_list[pair[0][0]].frag_list, F) # frags_para[色][frag]
        epi_cood_S, epi_cood_F = all_pa2co(frags_para)
        camL_idx = pair[0][1]
    elif pair[1] == "R":
        frags_para = epilines_para(cam_list[pair[0][1]].frag_list, F.T)
        epi_cood_S, epi_cood_F = all_pa2co(frags_para)
        camL_idx = pair[0][0]

    im_list = []
    for pair_col, col_part, epi_S_col, epi_F_col in zip(pair_list[pair], cam_list[camL_idx].frag_list, epi_cood_S, epi_cood_F):
        col_list=[]
        for pair_frag, epi_S_frag, epi_F_frag in zip(pair_col, epi_S_col, epi_F_col):
            pts = col_part[pair_frag] # 対応するフラグメント
            v1 = epi_F_frag - epi_S_frag
            v1_n = (v1[:,0]**2+v1[:,1]**2)**(1/2)
            v1_n = np.stack([v1_n, v1_n], axis=1)
            v1 = v1/v1_n
            v1_bro = np.array([np.broadcast_to(aa, (len(pts), 2)) for aa in v1])
            epi_cood_S_bro = np.array([np.broadcast_to(aa, (len(pts), 2)) for aa in epi_S_frag])
            v2 = pts - epi_cood_S_bro
            v2_n = (v2[:,:,0]**2+v2[:,:,1]**2)**(1/2)
            v2_n = np.stack([v2_n, v2_n], axis=2)
            v2 = v2/v2_n
            con_det = np.cross(v1_bro, v2)
            col_list.append(np.where(np.abs(con_det) <= 0.001))
        im_list.append(col_list)
        
    return im_list

In [17]:
# 点と線の衝突判定
coll_dict = {}
for pair in pair_list:
    im_list = PL_coll(pair, pair_list, cam_list)
    coll_dict[pair] = im_list

In [18]:
def pt_pair(coll_list):
    pool_i = []
    pool_j = []
    pre_i = None
    pre_j = None
    pt = 1
    for i, j in zip(coll_list[0], coll_list[1]):
        if i in pool_i:
            if pt == 1:
                continue
            elif pt == 0:
                if j not in pool_j:
                    pool_i.pop()
                    pool_j.pop()
                    pool_i.append(i)
                    pool_j.append(j)
                else:
                    continue

        elif i not in pool_i:
            if j in pool_j:
                pt = 0
            else:
                pt = 1
            pool_i.append(i)
            pool_j.append(j)
    return np.array([pool_i, pool_j])

In [19]:
# 点の一対一対応
pair_pt = {}
for i in coll_dict:
    im_list = []
    for col in coll_dict[i]:
        col_list = []
        for frag in col:
            newPair = pt_pair(frag)
            col_list.append(newPair)
        im_list.append(col_list)
    pair_pt[i] = im_list

In [20]:
def FR_frags(dict_tag):
    if dict_tag[1] == "F":
        part = cam_list[dict_tag[0][0]].frag_list
        counterpart = cam_list[dict_tag[0][1]].frag_list
        return part, counterpart
        
    elif i[1] == "R":
        part = cam_list[dict_tag[0][1]].frag_list
        counterpart = cam_list[dict_tag[0][0]].frag_list
        return part, counterpart

In [21]:
# 座標でdictを作る
coordinate_dict = {}
for i in pair_list:
    pair_coordinate = []
    part, counterpart = FR_frags(i)
    for part_col, cpart_col, pair_col, PtPair_col in zip(part, counterpart, pair_list[i], pair_pt[i]):
        col_list = []
        for part_frag, pair, pt_idx in zip(part_col, pair_col, PtPair_col):
            if pt_idx[0].size != 0:
                col_list.append((np.array([part_frag[pt_idx[0]], cpart_col[pair][pt_idx[1]]])))
        pair_coordinate.append(col_list)
    coordinate_dict[i] = pair_coordinate

In [22]:
def FR_check(dict_tag):
    if dict_tag[1] == "F":
        P1 = cam_list[dict_tag[0][0]].P
        P2 = cam_list[dict_tag[0][1]].P
        F = cam_pairs_F[dict_tag[0]]
        return P1, P2, F
    elif dict_tag[1] == "R":
        P1 = cam_list[dict_tag[0][1]].P
        P2 = cam_list[dict_tag[0][0]].P
        F = cam_pairs_F[dict_tag[0]].T
        return P1, P2, F

In [23]:
def connect_points(pts_list):
    sep_list = []
    root = np.transpose(pts_list[0], (1, 0, 2))
    length = len(root)-1
    sep_list.append(length)
    for i, pt in enumerate(pts_list):
        pt = np.transpose(pt, (1, 0, 2))
        if i == 0:
            continue
        root = np.concatenate([root, pt], 0)
        length += len(pt)
        sep_list.append(length)
    return root, sep_list

def sep_array(tri_pts, sep_list):
    # 一つの輪郭に対し削除リストから削除
    start = 0
    newArray = []
    for d in sep_list:
        if tri_pts[start:d].size != 0:
            newArray.append(tri_pts[start:d])
        start = d

    if tri_pts[start:].size != 0:
        newArray.append(tri_pts[start:])
    return newArray

TDlines = []
for i, j in enumerate(coordinate_dict):
    
    pts = coordinate_dict[j]
    P1_ori, P2_ori, F_ori = FR_check(j)
    #pt, sep_list = connect_points(pts)
    temp_TDlines = []
    for pt in pts:
        pt = np.transpose(pt, (1, 0, 2))
        F = np.broadcast_to(F_ori, (pt.shape[0],3,3))
        P1 = np.broadcast_to(P1_ori, (pt.shape[0],3,4))
        P2 = np.broadcast_to(P2_ori, (pt.shape[0],3,4))
        newcoords= np.array(list(map(min_dist, F, pt[:,1,:], pt[:,0,:])))
        tri_pt = np.array(list(map(tri, P1, P2, newcoords[:,1,:], newcoords[:,0,:])))
        #pts_array = sep_array(tri_pt, sep_list)
        temp_TDlines.append(tri_pt)
    TDlines.append(temp_TDlines)
    print((i+1)/len(coordinate_dict)*100)
    break

In [24]:
TDlines = {}
for i, j in enumerate(coordinate_dict):
    
    pts = coordinate_dict[j]
    P1_ori, P2_ori, F_ori = FR_check(j)
    #pt, sep_list = connect_points(pts)
    temp_TDlines = []
    for pts_col in pts:
        col_list = []
        for pt in pts_col:
            pt = np.transpose(pt, (1, 0, 2))
            F = np.broadcast_to(F_ori, (pt.shape[0],3,3))
            P1 = np.broadcast_to(P1_ori, (pt.shape[0],3,4))
            P2 = np.broadcast_to(P2_ori, (pt.shape[0],3,4))
            newcoords= np.array(list(map(min_dist, F, pt[:,1,:], pt[:,0,:])))
            tri_pt = np.array(list(map(tri, P1, P2, newcoords[:,1,:], newcoords[:,0,:])))
            #pts_array = sep_array(tri_pt, sep_list)
            col_list.append(tri_pt)
        temp_TDlines.append(col_list)
    TDlines[j] = temp_TDlines
    print((i+1)/len(coordinate_dict)*100)


2.083333333333333
4.166666666666666
6.25
8.333333333333332
10.416666666666668
12.5
14.583333333333334
16.666666666666664
18.75
20.833333333333336
22.916666666666664
25.0
27.083333333333332
29.166666666666668
31.25
33.33333333333333
35.41666666666667
37.5
39.58333333333333
41.66666666666667
43.75
45.83333333333333
47.91666666666667
50.0
52.083333333333336
54.166666666666664
56.25
58.333333333333336
60.416666666666664
62.5
64.58333333333334
66.66666666666666
68.75
70.83333333333334
72.91666666666666
75.0
77.08333333333334
79.16666666666666
81.25
83.33333333333334
85.41666666666666
87.5
89.58333333333334
91.66666666666666
93.75
95.83333333333334
97.91666666666666
100.0


In [25]:
def excluded_Parray(ex_tag):
    P_dict = {}
    for i, cam in enumerate(cam_list):
        if i in ex_tag:
            continue
        P_dict[i] = cam.P
    return P_dict

def dot_P_frag(P, frag):
    repro_frag = []
    for pt in frag:
        repro_pt = np.dot(P, pt)
        repro_pt = np.array(normalization(repro_pt))
        repro_frag.append(repro_pt)
    return np.array(repro_frag)

In [26]:
reprojection_dict = {}
for tag in TDlines:
    temp_reprojection_dict = {}
    P_dict = excluded_Parray(tag[0])
    for P_tag in P_dict:
        P = P_dict[P_tag]
        P_list = []
        for col in TDlines[tag]:
            col_list = []
            for i, frag in enumerate(col):
                frag = frag.reshape((-1,3))
                frag = np.concatenate([frag, np.ones(len(frag)).reshape((len(frag), 1))],1) # 末尾に1を追加 (X, Y, Z, 1)
                reprojection = dot_P_frag(P, frag)
                col_list.append(reprojection)
            P_list.append(col_list)
        temp_reprojection_dict[P_tag] = P_list
    reprojection_dict[tag] = temp_reprojection_dict

In [27]:
def connect_contour(contour_list):
    con_list = []
    for col in contour_list:
        A = np.concatenate(col).reshape((-1, 2))
        con_list.append(A)
    return con_list

In [29]:
def cal_distance(repro_P, contour_P):
    contour_P = connect_contour(contour_P)
    distance_list = []
    for repro_col, con_col in zip(repro_P, contour_P):
        col_list = []
        for repro_frag in repro_col:
            repro_frag_bro = np.array([np.broadcast_to(aa, (len(con_col), 2)) for aa in repro_frag])
            distance = (np.sum((con_col - repro_frag_bro)**2,axis=2))**(1/2)
            col_list.append(distance)
        distance_list.append(col_list)
    return distance_list

def distance_check(distance_list):
    dist_check_list = []
    ac_list = []
    for col in distance_list:
        col_list = []
        ac_col_list = []
        for frag in col:
            ac = np.array((np.min(frag,axis=1)) < 10,dtype=np.int64) # 条件:10 pixel以内
            col_list.append(sum(ac)/len(ac))
            ac_col_list.append(ac)
        ac_list.append(ac_col_list)
        dist_check_list.append(np.array(col_list))
    return ac_list, dist_check_list

def P_dict_check(repro_dict_taged):
    P_list = []
    P_ac_list = []
    for P_tag in repro_dict_taged:
        repro_P = repro_dict_taged[P_tag]
        contour_P = cam_list[P_tag].contour_list
        distance_list = cal_distance(repro_P, contour_P)
        ac_list, dist_check_list = distance_check(distance_list)
        P_list.append(dist_check_list)
        P_ac_list.append(ac_list)
    P_check = np.array(P_list)
    return ac_list, P_ac_list, P_check

def P_check_integration(P_check):
    check_list = []
    for col in range(P_check.shape[1]):
        temp_list = []
        for img in P_check[:,col]:
            temp = np.array(img > 0.5,dtype=np.int64)# 曲線中の何割が閾値以内か
            temp_list.append(temp)
        col_check = np.sum(np.array(temp_list),axis=0)
        check_list.append(col_check)
    return check_list

def ac_list_integration(P_ac_list):
    inter_ac = []
    for i, col in enumerate(P_ac_list[0]):
        col_list = []
        for j in range(len(col)):
            temp_array = np.zeros(len(P_ac_list[0][i][j]))
            for img in P_ac_list:
                temp_array += img[i][j]
            col_list.append(temp_array)
        inter_ac.append(col_list)
    return inter_ac
    
def gen_support_dict(reprojection_dicte):
    support_dict = {}
    for tag in reprojection_dict:
        repro_dict_taged = reprojection_dict[tag]
        P_ac_list, P_check = P_dict_check(repro_dict_taged)
        check_list = P_check_integration(P_check)
        inter_ac = ac_list_integration(P_ac_list)
        support_dict[tag] = (check_list, inter_ac)
    return support_dict

def get_tilt(curve):
    tilt_list = []
    for i in renge(curve):
        if i == 0:
            tilt = (curve[i]-curve[i+1])
            
        elif i == len(curve) - 1:
            tilt = (curve[i-1]-curve[i])
            
        else:
            tilt = (curve[i-1]-curve[i+1])/2
        tilt_list.append(tilt)
    return np.array(tilt_list)

def tilt_comparison(tilt1, tilt2):
    angle1 = np.arctan(tilt1)
    angle2 = np.arctan(tilt2)
    return np.abs(angle1-angle2)

def tilt_filter(curve1, curve2):
    tilt1 = get_tilt(curve1)
    tilt2 = get_tilt(curve2)
    tilt_com_list = tilt_comparison(tilt1, tilt2)
    return tilt_com_list


In [ ]:
def 

In [30]:
support_dict = gen_support_dict(reprojection_dict)

In [31]:
sup_th = 11 # サポート数
curve_fragment = []
for tag in TDlines:
    if tag[1] == "R":
        continue
    lines_list = TDlines[tag]
    support_list, support_ac = support_dict[tag][0], support_dict[tag][1]
    for col, sup_col, ac_col in zip(lines_list, support_list, support_ac):
        for frag, sup, sup_ac in zip(col, sup_col, ac_col):
            if sup > sup_th:
                frag = np.reshape(frag,(-1, 3))
                frag = np.array([i for i,j in zip(frag, sup_ac >sup_th) if j])
                curve_fragment.append(frag)

In [33]:
support_ac[5]

[array([1., 1., 2., 1., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]),
 array([17., 21., 21., 21., 21., 19., 12.,  8., 21., 21., 21., 21., 21.,
        21., 21., 21., 21., 21., 21., 12., 21.,  8.,  8., 21., 12., 16.,
        21., 21., 21., 13.,  6., 21., 11.,  8.,  2.,  2., 21., 20., 20.,
        20., 20., 20., 20., 21., 20., 21., 21., 21., 21., 19., 22., 22.,
        22., 22., 22., 21., 21., 21., 20., 21., 21., 21., 21., 20., 20.,
        20., 20., 20., 20., 19., 19., 19., 19., 19., 19., 19., 19., 19.,
        18., 18.,  8., 19., 18., 15.]),
 array([20., 20., 20., 20., 21., 20., 21., 21., 21., 21., 19., 22., 22.,
        22., 22., 22., 21., 21., 21., 20., 21., 21., 21., 21., 20., 20.,
        20., 20., 20., 20., 19., 19., 19., 19., 19., 19., 19., 19., 19.,
        18., 18.,  8., 19., 18., 15., 18., 19., 11., 18., 18., 18., 18.,
        18., 18., 18., 18.]),
 array([20., 21., 

In [32]:
ac_col

[array([16., 16., 16., 16., 16., 16., 16., 16., 16., 17., 17., 17., 18.,
        18., 18., 18., 19., 19., 19., 19., 19., 19., 19.]),
 array([16., 16., 16., 16., 16., 16., 16., 16., 16., 17., 17., 17., 18.,
        18., 18., 18., 19., 19., 19., 19., 19., 19., 19., 18.]),
 array([16., 16., 16., 16., 16., 16., 16., 16., 16., 17., 17., 17., 18.,
        18., 18., 18., 19., 19., 19., 19., 19., 19., 19.]),
 array([17., 18., 18., 18., 16., 17., 16., 16., 16., 16., 13., 13., 14.,
        14., 14., 14., 14., 14.]),
 array([16., 16., 16., 13., 13., 14., 14., 14., 14., 14., 14.]),
 array([16., 16., 16., 16., 13., 13., 14., 14., 14., 14., 14., 14.]),
 array([17., 17., 17., 17., 17., 17., 17., 17., 17., 17., 17., 17., 17.,
        17., 17., 17., 17., 17., 14., 14., 16., 17., 13., 17., 17., 17.,
        13., 13., 15., 15., 15., 15., 13., 14., 15., 16., 17., 13., 13.,
        13., 13., 13., 13., 17., 17., 17., 17., 17., 17., 17., 17., 17.,
        17., 17.]),
 array([17., 17., 17., 17., 17., 17., 17.

from scipy import interpolate
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook

fig = plt.figure(figsize = (12, 12))
fig.patch.set_alpha(0.)
# 3DAxesを追加
ax = fig.add_subplot(111, projection='3d')

# 軸ラベルを設定
ax.set_xlabel("x", size = 14)
ax.set_ylabel("z", size = 14)
ax.set_zlabel("y", size = 14)
ax.set_box_aspect((1,1,1))

def plot_graph():
    for frag in curve_fragment:
        if frag.size == 0:
            continue
        #print(frag)
        x = frag[:,0]
        y = frag[:,1]
        z = frag[:,2]
        data = [x,y,z]
        try:
            tck, u= interpolate.splprep(data, k=3)
        except ValueError:
            pass
        except TypeError:
            pass
        new = interpolate.splev(u, tck, der=0)

        ax.plot(new[0], new[2], new[1],"-")

def plt_graph3d(angle):
    ax.view_init(azim=angle*5)
    
# アニメーションを作成
ani = FuncAnimation(
    fig,
    func=plt_graph3d,
    frames=72,
    init_func=plot_graph,
    interval=200
)

# imagemagickで作成したアニメーションをGIFで書き出す
ani.save("rolling.gif", writer="pillow", savefig_kwargs={'transparent': True, 'facecolor': 'none'})
ani.save('anim.mp4', writer="ffmpeg", savefig_kwargs={'transparent': True, 'facecolor': 'none'})

In [ ]:
from scipy import interpolate
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook

def plot_graph():
    for frag in curve_fragment:
        if frag.size == 0:
            continue
        #print(frag)
        x = frag[:,0]
        y = frag[:,1]
        z = frag[:,2]
        data = [x,y,z]
        try:
            tck, u= interpolate.splprep(data, k=3)
        except ValueError:
            pass
        except TypeError:
            pass
        new = interpolate.splev(u, tck, der=0)

        ax.plot(new[0], new[2], new[1],"-")
        #ax.plot(x, z, y,"-")
%matplotlib notebook
fig = plt.figure(figsize = (12, 12))
fig.patch.set_alpha(0.)
# 3DAxesを追加
ax = fig.add_subplot(111, projection='3d')

# 軸ラベルを設定
ax.set_xlabel("x", size = 14)
ax.set_ylabel("z", size = 14)
ax.set_zlabel("y", size = 14)
ax.set_box_aspect((1,1,1))
#ax.set_xticks(np.arange(-4,4,1))
#ax.set_yticks(np.arange(-4,4,1))
#ax.set_zticks(np.arange(-2,9,1))

plot_graph()